# CAMEL 角色扮演自主合作代理

这是一篇关于“CAMEL: Communicative Agents for “Mind” Exploration of Large Scale Language Model Society”论文的langchain实现。

概述：

会话和基于聊天的语言模型的快速发展已经在复杂任务解决方面取得了显著进展。然而，它们的成功在很大程度上依赖于人类输入来引导对话，这可能具有挑战性且耗时。本文探讨了构建可扩展技术以促进沟通代理之间的自主合作并提供对其“认知”过程的洞察力的潜力。为了解决实现自主合作的挑战，我们提出了一种名为角色扮演的新型沟通代理框架。我们的方法涉及使用启示式提示来引导聊天代理完成任务，同时保持与人类意图的一致性。我们展示了如何使用角色扮演来生成用于研究聊天代理的行为和能力的对话数据，为研究会话语言模型提供了宝贵的资源。我们的贡献包括引入一种新颖的沟通代理框架，提供一种可扩展的方法来研究多代理系统的合作行为和能力，并开源我们的库以支持对沟通代理及其他领域的研究。

原始实现：https://github.com/lightaime/camel

项目网站：https://www.camel-ai.org/

Arxiv论文：https://arxiv.org/abs/2303.17760

## 导入与 LangChain 相关的模块

In [1]:
from typing import List

from langchain.prompts.chat import (
    HumanMessagePromptTemplate,
    SystemMessagePromptTemplate,
)
from langchain.schema import (
    AIMessage,
    BaseMessage,
    HumanMessage,
    SystemMessage,
)
from langchain_openai import ChatOpenAI

## 定义一个CAMEL代理助手类

In [2]:
class CAMELAgent:
    def __init__(
        self,
        system_message: SystemMessage,
        model: ChatOpenAI,
    ) -> None:
        self.system_message = system_message
        self.model = model
        self.init_messages()

    def reset(self) -> None:
        self.init_messages()
        return self.stored_messages

    def init_messages(self) -> None:
        self.stored_messages = [self.system_message]

    def update_messages(self, message: BaseMessage) -> List[BaseMessage]:
        self.stored_messages.append(message)
        return self.stored_messages

    def step(
        self,
        input_message: HumanMessage,
    ) -> AIMessage:
        messages = self.update_messages(input_message)

        output_message = self.model.invoke(messages)
        self.update_messages(output_message)

        return output_message

class CAMELAgent:
    def __init__(
        self,
        system_message: SystemMessage,  # 初始化CAMELAgent类的系统消息参数
        model: ChatOpenAI,  # 初始化CAMELAgent类的模型参数
    ) -> None:
        self.system_message = system_message  # 将系统消息参数赋值给类的属性
        self.model = model  # 将模型参数赋值给类的属性
        self.init_messages()  # 调用init_messages()方法初始化消息列表

    def reset(self) -> None:
        self.init_messages()  # 重置消息列表
        return self.stored_messages  # 返回重置后的消息列表

    def init_messages(self) -> None:
        self.stored_messages = [self.system_message]  # 初始化消息列表，将系统消息添加到列表中

    def update_messages(self, message: BaseMessage) -> List[BaseMessage]:
        self.stored_messages.append(message)  # 将新消息添加到消息列表中
        return self.stored_messages  # 返回更新后的消息列表

    def step(
        self,
        input_message: HumanMessage,
    ) -> AIMessage:
        messages = self.update_messages(input_message)  # 更新消息列表，将用户输入的消息添加到列表中

        output_message = self.model.invoke(messages)  # 使用模型生成回复消息
        self.update_messages(output_message)  # 将回复消息添加到消息列表中

        return output_message  # 返回生成的回复消息

## 设置OpenAI API密钥以及角色和角色扮演任务

In [3]:
# 导入os模块
import os

# 设置OpenAI API密钥为空字符串
os.environ["OPENAI_API_KEY"] = ""

# 定义助手角色名称为"Python Programmer"
assistant_role_name = "Python Programmer"

# 定义用户角色名称为"Stock Trader"
user_role_name = "Stock Trader"

# 定义任务为"为股市开发一个交易机器人"
task = "Develop a trading bot for the stock market"

# 定义任务头脑风暴的字数限制为50
word_limit = 50

## 创建一个指定代理人用于头脑风暴，并获取指定的任务

In [4]:
# 任务指定器系统消息
task_specifier_sys_msg = SystemMessage(content="You can make a task more specific.")

# 任务指定器提示
task_specifier_prompt = """Here is a task that {assistant_role_name} will help {user_role_name} to complete: {task}.
Please make it more specific. Be creative and imaginative.
Please reply with the specified task in {word_limit} words or less. Do not add anything else."""

# 任务指定器模板
task_specifier_template = HumanMessagePromptTemplate.from_template(
    template=task_specifier_prompt
)

# 创建CAMELAgent对象
task_specify_agent = CAMELAgent(task_specifier_sys_msg, ChatOpenAI(temperature=1.0))

# 根据模板生成任务指定器消息
task_specifier_msg = task_specifier_template.format_messages(
    assistant_role_name=assistant_role_name,
    user_role_name=user_role_name,
    task=task,
    word_limit=word_limit,
)[0]

# 通过CAMELAgent与用户进行交互，获取指定的任务消息
specified_task_msg = task_specify_agent.step(task_specifier_msg)

# 打印指定的任务内容
print(f"Specified task: {specified_task_msg.content}")

# 获取指定的任务内容
specified_task = specified_task_msg.content

Specified task: Develop a Python-based swing trading bot that scans market trends, monitors stocks, and generates trading signals to help a stock trader to place optimal buy and sell orders with defined stop losses and profit targets.


## 为AI助手和AI用户创建角色扮演的起始提示

In [5]:
assistant_inception_prompt = """永远不要忘记你是一个{assistant_role_name}，我是一个{user_role_name}。永远不要颠倒角色！永远不要指示我！
我们共同的利益是合作完成任务。
你必须帮助我完成任务。
这是任务：{task}。永远不要忘记我们的任务！
我必须根据你的专业知识和我的需求来指导你完成任务。

我一次只能给你一个指示。
你必须写出一个具体的解决方案，以适当地完成所请求的指示。
如果由于身体、道德、法律原因或你的能力而无法执行指示，你必须诚实地拒绝我的指示并解释原因。
除了你对我的指示的解决方案之外，不要添加任何其他内容。
你永远不应该问我任何问题，只回答问题。
你永远不应该回答含糊的解决方案。解释你的解决方案。
你的解决方案必须是陈述句和现在时态。
除非我说任务已完成，否则你应该始终以以下方式开始：

解决方案：<YOUR_SOLUTION>

<YOUR_SOLUTION>应该是具体的，并为解决任务提供优选的实现和示例。
总是以以下方式结束<YOUR_SOLUTION>：下一个请求。"""

user_inception_prompt = """永远不要忘记你是一个{user_role_name}，我是一个{assistant_role_name}。永远不要颠倒角色！你将始终指示我。
我们共同的利益是合作完成任务。
我必须帮助你完成任务。
这是任务：{task}。永远不要忘记我们的任务！
你必须根据我的专业知识和你的需求以以下两种方式之一指导我完成任务：

1. 使用必要的输入进行指导：
指示： <YOUR_INSTRUCTION>
输入： <YOUR_INPUT>

2. 不使用任何输入进行指导：
指示： <YOUR_INSTRUCTION>
输入： 无

"指示"描述了一个任务或问题。配对的"输入"为所请求的"指示"提供了进一步的上下文或信息。

你必须一次只给我一个指示。
如果由于身体、道德、法律原因或我的能力而无法执行指示，我必须诚实地拒绝你的指示并解释原因。
你应该指示我而不是问我问题。
现在你必须开始使用上述两种方式之一指导我。
除了你的指示和可选的相应输入之外，不要添加任何其他内容！
继续给我指示和必要的输入，直到你认为任务已完成。
当任务完成时，你必须只回复一个单词<CAMEL_TASK_DONE>。
除非我的回复解决了你的任务，否则永远不要说<CAMEL_TASK_DONE>。"""

## 创建一个辅助函数，通过角色名称和任务来获取AI助手和AI用户的系统消息。

In [6]:
def get_sys_msgs(assistant_role_name: str, user_role_name: str, task: str):
    # 创建助手系统消息模板
    assistant_sys_template = SystemMessagePromptTemplate.from_template(
        template=assistant_inception_prompt
    )
    # 格式化助手系统消息
    assistant_sys_msg = assistant_sys_template.format_messages(
        assistant_role_name=assistant_role_name,
        user_role_name=user_role_name,
        task=task,
    )[0]

    # 创建用户系统消息模板
    user_sys_template = SystemMessagePromptTemplate.from_template(
        template=user_inception_prompt
    )
    # 格式化用户系统消息
    user_sys_msg = user_sys_template.format_messages(
        assistant_role_name=assistant_role_name,
        user_role_name=user_role_name,
        task=task,
    )[0]

    # 返回助手系统消息和用户系统消息
    return assistant_sys_msg, user_sys_msg

## 从获取的系统消息中创建AI助手代理和AI用户代理

In [7]:
# 获取系统消息
assistant_sys_msg, user_sys_msg = get_sys_msgs(assistant_role_name, user_role_name, specified_task)

# 创建助手代理
assistant_agent = CAMELAgent(assistant_sys_msg, ChatOpenAI(temperature=0.2))

# 创建用户代理
user_agent = CAMELAgent(user_sys_msg, ChatOpenAI(temperature=0.2))

# 重置代理
assistant_agent.reset()
user_agent.reset()

# 初始化对话
user_msg = HumanMessage(
    content=(
        f"{user_sys_msg.content}. "
        "现在开始逐个给我介绍。"
        "只回答指令和输入。"
    )
)

assistant_msg = HumanMessage(content=f"{assistant_sys_msg.content}")
assistant_msg = assistant_agent.step(user_msg)

## 开始角色扮演会话以解决任务！

In [8]:
# 打印原始任务提示
print(f"原始任务提示：\n{task}\n")
# 打印指定的任务提示
print(f"指定的任务提示：\n{specified_task}\n")

# 设定对话轮次限制和初始轮次
chat_turn_limit, n = 30, 0
while n < chat_turn_limit:
    n += 1
    # 用户AI发送消息
    user_ai_msg = user_agent.step(assistant_msg)
    user_msg = HumanMessage(content=user_ai_msg.content)
    # 打印用户AI消息
    print(f"AI 用户 ({user_role_name}):\n\n{user_msg.content}\n\n")

    # 助手AI回复消息
    assistant_ai_msg = assistant_agent.step(user_msg)
    assistant_msg = HumanMessage(content=assistant_ai_msg.content)
    # 打印助手AI消息
    print(f"AI 助手 ({assistant_role_name}):\n\n{assistant_msg.content}\n\n")
    # 如果用户消息中包含"<CAMEL_TASK_DONE>"，则跳出循环
    if "<CAMEL_TASK_DONE>" in user_msg.content:
        break

Original task prompt:
Develop a trading bot for the stock market

Specified task prompt:
Develop a Python-based swing trading bot that scans market trends, monitors stocks, and generates trading signals to help a stock trader to place optimal buy and sell orders with defined stop losses and profit targets.

AI User (Stock Trader):

Instruction: Install the necessary Python libraries for data analysis and trading.
Input: None


AI Assistant (Python Programmer):

Solution: We can install the necessary Python libraries using pip, a package installer for Python. We can install pandas, numpy, matplotlib, and ta-lib for data analysis and trading. We can use the following command to install these libraries:

```
pip install pandas numpy matplotlib ta-lib
```

Next request.


AI User (Stock Trader):

Instruction: Import the necessary libraries in the Python script.
Input: None


AI Assistant (Python Programmer):

Solution: We can import the necessary libraries in the Python script using the im